In [1]:
import pandas as pd
import gmaps
import requests

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
file = "Resources/listings_cleaned2.csv"

In [3]:
region_df = pd.read_csv(file)
region_df.head()

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,zipcode,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_days_active,time_since_first_review,time_since_last_review,listing_url
0,958.0,7/31/08,within a few hours,100%,1.0,1.0,1.0,1.0,Western Addition,94115.0,...,1.0,1.0,1.0,0.0,0.0,1.60,3958.0,4+ years,17.0,https://www.airbnb.com/rooms/958
1,7918.0,6/17/09,within a few hours,100%,0.0,10.0,10.0,1.0,Haight Ashbury,94117.0,...,0.0,9.0,0.0,9.0,0.0,0.14,3637.0,4+ years,923.0,https://www.airbnb.com/rooms/7918
2,8142.0,6/17/09,within a few hours,100%,0.0,10.0,10.0,1.0,Haight Ashbury,94117.0,...,0.0,9.0,0.0,9.0,0.0,0.14,3637.0,4+ years,263.0,https://www.airbnb.com/rooms/8142
3,8339.0,7/2/09,within an hour,100%,0.0,2.0,2.0,1.0,Western Addition,94115.0,...,0.0,2.0,2.0,0.0,0.0,0.23,3622.0,4+ years,295.0,https://www.airbnb.com/rooms/8339
4,8739.0,1/27/09,within an hour,100%,1.0,2.0,2.0,0.0,Mission,94110.0,...,1.0,2.0,0.0,2.0,0.0,5.61,3778.0,4+ years,12.0,https://www.airbnb.com/rooms/8739


In [4]:
region_df.dtypes

id                                              float64
host_since                                       object
host_response_time                               object
host_response_rate                               object
host_is_superhost                               float64
host_listings_count                             float64
host_total_listings_count                       float64
host_identity_verified                          float64
neighbourhood_cleansed                           object
zipcode                                         float64
latitude                                        float64
longitude                                       float64
property_type                                    object
room_type                                        object
accommodates                                    float64
bathrooms                                       float64
bedrooms                                        float64
beds                                            

In [5]:
# Filter hosts and
clean_df = region_df.loc[(region_df["host_response_rate"] == '100%') & 
                        (region_df['host_is_superhost'] == 1) &
                        (region_df['host_identity_verified'] == 1) &
                        (region_df['instant_bookable'] == 1) & 
                        # review ratings.
                        (region_df['review_scores_rating'] >= 80) & 
                        (region_df['review_scores_accuracy'] >= 7) & 
                        (region_df['review_scores_cleanliness'] >= 7) & 
                        (region_df['review_scores_checkin'] >= 7) & 
                        (region_df['review_scores_communication'] >= 7) & 
                        (region_df['review_scores_location'] >= 7) & 
                        (region_df['review_scores_value'] >= 7) &
                        # recent calendar updates.
                        (region_df['calendar_updated'] != '17 months ago') &
                        (region_df['calendar_updated'] != '14 months ago') &
                        (region_df['calendar_updated'] != '12 months ago')]
clean_df

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,zipcode,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_days_active,time_since_first_review,time_since_last_review,listing_url
0,958.0,7/31/08,within a few hours,100%,1.0,1.0,1.0,1.0,Western Addition,94115.0,...,1.0,1.0,1.0,0.0,0.0,1.60,3958.0,4+ years,17.0,https://www.airbnb.com/rooms/958
11,12522.0,5/22/09,within an hour,100%,1.0,1.0,1.0,1.0,Castro/Upper Market,94110.0,...,1.0,1.0,0.0,1.0,0.0,3.48,3663.0,4+ years,18.0,https://www.airbnb.com/rooms/12522
18,23611.0,5/25/09,within an hour,100%,1.0,3.0,3.0,1.0,Bernal Heights,94110.0,...,1.0,2.0,1.0,1.0,0.0,2.17,3660.0,4+ years,9.0,https://www.airbnb.com/rooms/23611
48,51557.0,9/16/10,within an hour,100%,1.0,1.0,1.0,1.0,Western Addition,94115.0,...,1.0,1.0,1.0,0.0,0.0,1.00,3181.0,4+ years,78.0,https://www.airbnb.com/rooms/51557
58,59702.0,11/14/10,within an hour,100%,1.0,1.0,1.0,1.0,Bernal Heights,94110.0,...,1.0,1.0,0.0,1.0,0.0,1.93,3122.0,4+ years,30.0,https://www.airbnb.com/rooms/59702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4924,32744842.0,5/5/14,within an hour,100%,1.0,1.0,1.0,1.0,Outer Richmond,94121.0,...,1.0,1.0,1.0,0.0,0.0,1.70,1854.0,0-6 months,41.0,https://www.airbnb.com/rooms/29610726
4925,32746244.0,6/14/14,within an hour,100%,1.0,1.0,1.0,1.0,Outer Richmond,94121.0,...,1.0,1.0,1.0,0.0,0.0,7.76,1814.0,0-6 months,2.0,https://www.airbnb.com/rooms/29611738
4963,33166972.0,2/15/12,within an hour,100%,1.0,4.0,4.0,1.0,Castro/Upper Market,94110.0,...,1.0,4.0,1.0,2.0,1.0,12.35,2664.0,0-6 months,3.0,https://www.airbnb.com/rooms/29854300
5012,34056383.0,7/8/11,within an hour,100%,1.0,2.0,2.0,1.0,Outer Sunset,94132.0,...,1.0,2.0,2.0,0.0,0.0,1.00,2886.0,0-6 months,16.0,https://www.airbnb.com/rooms/30157169


In [6]:
clean_df.calendar_updated.value_counts()

today            108
2 weeks ago       86
3 weeks ago       37
a week ago        35
4 days ago        27
yesterday         26
4 weeks ago       24
2 months ago      21
3 days ago        19
5 weeks ago       19
5 days ago        19
6 days ago        13
6 weeks ago       12
7 weeks ago       12
2 days ago        10
1 week ago         9
3 months ago       7
7 months ago       3
4 months ago       3
10 months ago      2
5 months ago       2
8 months ago       1
6 months ago       1
11 months ago      1
Name: calendar_updated, dtype: int64

In [7]:
# Choose a minimum and maximum price values.
min_price = float(input("What is the minimum price you would like for the listing? "))
max_price = float(input("What is the maximum price you would like for the listing? "))

What is the minimum price you would like for the listing? 50
What is the maximum price you would like for the listing? 100


In [8]:
# Filter the dataset to find the listings that fit the criteria.
preferred_df = clean_df.loc[(clean_df["price"] <= max_price) & (clean_df["price"] >= min_price)]
preferred_df

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,zipcode,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_days_active,time_since_first_review,time_since_last_review,listing_url
11,12522.0,5/22/09,within an hour,100%,1.0,1.0,1.0,1.0,Castro/Upper Market,94110.0,...,1.0,1.0,0.0,1.0,0.0,3.48,3663.0,4+ years,18.0,https://www.airbnb.com/rooms/12522
18,23611.0,5/25/09,within an hour,100%,1.0,3.0,3.0,1.0,Bernal Heights,94110.0,...,1.0,2.0,1.0,1.0,0.0,2.17,3660.0,4+ years,9.0,https://www.airbnb.com/rooms/23611
135,208727.0,4/19/11,within an hour,100%,1.0,8.0,8.0,1.0,Outer Richmond,94121.0,...,1.0,2.0,0.0,2.0,0.0,0.33,2966.0,4+ years,265.0,https://www.airbnb.com/rooms/208727
157,256176.0,8/16/11,within an hour,100%,1.0,1.0,1.0,1.0,Inner Richmond,94118.0,...,1.0,1.0,0.0,1.0,0.0,3.12,2847.0,4+ years,4.0,https://www.airbnb.com/rooms/256176
247,474107.0,5/8/12,within an hour,100%,1.0,1.0,1.0,1.0,Excelsior,94112.0,...,1.0,1.0,0.0,1.0,0.0,2.52,2581.0,1-2 years,11.0,https://www.airbnb.com/rooms/474107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4713,31206489.0,9/11/14,within an hour,100%,1.0,3.0,3.0,1.0,Noe Valley,94114.0,...,1.0,3.0,0.0,3.0,0.0,6.27,1725.0,0-6 months,4.0,https://www.airbnb.com/rooms/28079442
4762,31595144.0,4/3/12,within an hour,100%,1.0,4.0,4.0,1.0,Bernal Heights,94110.0,...,1.0,4.0,1.0,3.0,0.0,5.82,2616.0,0-6 months,1.0,https://www.airbnb.com/rooms/28407357
4835,32210241.0,1/29/12,within an hour,100%,1.0,4.0,4.0,1.0,Pacific Heights,94115.0,...,1.0,3.0,1.0,2.0,0.0,2.06,2681.0,0-6 months,20.0,https://www.airbnb.com/rooms/28920042
4894,32617586.0,3/4/13,within an hour,100%,1.0,6.0,6.0,1.0,Mission,94110.0,...,1.0,5.0,0.0,5.0,0.0,1.38,2281.0,0-6 months,28.0,https://www.airbnb.com/rooms/29385707


In [9]:
# Create a descirption box.
info_box_template = """
<dl>
<dt>ID</dt><dd>{id}</dd>
<dt>Neighbourhood</dt><dd>{neighbourhood_cleansed}</dd>
<dt>Price</dt><dd>{price}</dd>
<dt>Rating</dt><dd>{review_scores_rating}</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
listing_info = [info_box_template.format(**row) for index, row in preferred_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = preferred_df[["latitude", "longitude"]]

In [10]:
# Assign the figure variable.
fig = gmaps.figure(center=(37.77, -122.4), zoom_level=11.8)

# Add a marker_layer.
marker_layer = gmaps.marker_layer(locations, info_box_content=listing_info)
fig.add_layer(marker_layer)

# Display.
fig

Figure(layout=FigureLayout(height='420px'))